In [1]:
!pip install kaggle==1.5.12

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.0/59.0 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for kaggle: filename=kaggle-1.5.12-py3-none-any.whl size=73026 sha256=8f7607afd1287272b4a45b1487062c762f2b3377f0d3fa5b80151278bd75bd53
  Stored in directory: /root/.cache/pip/wheels/2e/27/39/f44e52756a6407b444143f233abe9fda0e18a23e8b20e0cd1c
Successfully built kaggle
  Attempting uninstall: kaggle
    Found existing installation: kaggle 1.7.4.2
    Uninstalling kaggle-1.7.4.2:
      Successfully uninstalled kaggle-1.7.4.2


In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json  # Set permissions

In [3]:
!mkdir datasets

In [4]:
!kaggle datasets download -d mozillaorg/common-voice -p /content/datasets --force

100% 12.0G/12.0G [02:35<00:00, 127MB/s] 
100% 12.0G/12.0G [02:35<00:00, 83.4MB/s]


In [5]:
%cd datasets

/content/datasets


In [6]:
!unzip common-voice.zip

Streaming output truncated to the last 5000 lines.
  inflating: cv-valid-train/cv-valid-train/sample-190776.mp3  
  inflating: cv-valid-train/cv-valid-train/sample-190777.mp3  
  inflating: cv-valid-train/cv-valid-train/sample-190778.mp3  
  inflating: cv-valid-train/cv-valid-train/sample-190779.mp3  
  inflating: cv-valid-train/cv-valid-train/sample-190780.mp3  
  inflating: cv-valid-train/cv-valid-train/sample-190781.mp3  
  inflating: cv-valid-train/cv-valid-train/sample-190782.mp3  
  inflating: cv-valid-train/cv-valid-train/sample-190783.mp3  
  inflating: cv-valid-train/cv-valid-train/sample-190784.mp3  
  inflating: cv-valid-train/cv-valid-train/sample-190785.mp3  
  inflating: cv-valid-train/cv-valid-train/sample-190786.mp3  
  inflating: cv-valid-train/cv-valid-train/sample-190787.mp3  
  inflating: cv-valid-train/cv-valid-train/sample-190788.mp3  
  inflating: cv-valid-train/cv-valid-train/sample-190789.mp3  
  inflating: cv-valid-train/cv-valid-train/sample-190790.mp3  
  in

In [7]:
import pandas as pd

In [8]:
dev_df = pd.read_csv('cv-valid-dev.csv')
dev_df.head()

,filename,text,up_votes,down_votes,age,gender,accent,duration
0,cv-valid-dev/sample-000000.mp3,be careful with your prognostications said the...,1,0,NaN,NaN,NaN,NaN
1,cv-valid-dev/sample-000001.mp3,then why should they be surprised when they se...,2,0,NaN,NaN,NaN,NaN
2,cv-valid-dev/sample-000002.mp3,a young arab also loaded down with baggage ent...,2,0,NaN,NaN,NaN,NaN
3,cv-valid-dev/sample-000003.mp3,i thought that everything i owned would be des...,3,0,NaN,NaN,NaN,NaN
4,cv-valid-dev/sample-000004.mp3,he moved about invisible but everyone could he...,1,0,fourties,female,england,NaN


In [9]:
# dev_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4076 entries, 0 to 4075
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   filename    4076 non-null   object 
 1   text        4076 non-null   object 
 2   up_votes    4076 non-null   int64  
 3   down_votes  4076 non-null   int64  
 4   age         1528 non-null   object 
 5   gender      1540 non-null   object 
 6   accent      1350 non-null   object 
 7   duration    0 non-null      float64
dtypes: float64(1), int64(2), object(5)
memory usage: 254.9+ KB


In [10]:
!pip install pydub

In [11]:
from IPython.display import Audio, display
import os

def display_audio(audio_path):
  display(Audio(audio_path))

audio_directory = 'cv-valid-dev/cv-valid-dev'
audio_files = [audio for audio in os.listdir(audio_directory)]

for audio_file in audio_files[:5]:
  audio_path = os.path.join(audio_directory, audio_file)
  display_audio(audio_path)


In [12]:
# dev_df['down_votes'].value_counts()

,count
down_votes,
0,3652
1,363
2,40
3,10
4,4
12,2
15,2
6,1
21,1


In [13]:
# dev_df[dev_df['down_votes'] == 0][:10]

,filename,text,up_votes,down_votes,age,gender,accent,duration
0,cv-valid-dev/sample-000000.mp3,be careful with your prognostications said the...,1,0,NaN,NaN,NaN,NaN
1,cv-valid-dev/sample-000001.mp3,then why should they be surprised when they se...,2,0,NaN,NaN,NaN,NaN
2,cv-valid-dev/sample-000002.mp3,a young arab also loaded down with baggage ent...,2,0,NaN,NaN,NaN,NaN
3,cv-valid-dev/sample-000003.mp3,i thought that everything i owned would be des...,3,0,NaN,NaN,NaN,NaN
4,cv-valid-dev/sample-000004.mp3,he moved about invisible but everyone could he...,1,0,fourties,female,england,NaN
5,cv-valid-dev/sample-000005.mp3,but everything had changed,3,0,teens,male,us,NaN
6,cv-valid-dev/sample-000006.mp3,are you sure this is claire,2,0,NaN,NaN,NaN,NaN
7,cv-valid-dev/sample-000007.mp3,it had told him to dig where his tears fell,1,0,NaN,NaN,NaN,NaN
8,cv-valid-dev/sample-000008.mp3,the shop folks were taking down their shutters...,1,0,twenties,female,canada,NaN
9,cv-valid-dev/sample-000009.mp3,the teacher thought that he'd taught himself a...,1,0,fifties,female,australia,NaN


In [14]:
#

 => It seems that the difference between upvotes and downvotes doesn't relate to the quality of audios

**Preprocessing steps**



In [15]:
dev_df = dev_df.drop(columns=dev_df.columns[dev_df.columns.get_loc('up_votes') : dev_df.columns.get_loc('duration') + 1])

In [16]:
dev_df.head()

,filename,text
0,cv-valid-dev/sample-000000.mp3,be careful with your prognostications said the...
1,cv-valid-dev/sample-000001.mp3,then why should they be surprised when they se...
2,cv-valid-dev/sample-000002.mp3,a young arab also loaded down with baggage ent...
3,cv-valid-dev/sample-000003.mp3,i thought that everything i owned would be des...
4,cv-valid-dev/sample-000004.mp3,he moved about invisible but everyone could he...


In [17]:
import torch
import torchaudio
import warnings
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')

In [18]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [19]:
train_df, test_df = torch.utils.data.random_split(dev_df, [0.8, 0.2])

In [20]:
def preprocess_for_rnnt_torchaudio(file_path, target_sr=16000, max_duration=5):
    """GPU-accelerated preprocessing pipeline for RNN-T using TorchAudio"""

    # 1. Load audio directly to GPU (PyTorch 2.0+)
    waveform, sr = torchaudio.load(file_path)
    waveform = waveform.to(device)

    # 2. Resample if needed
    if sr != target_sr:
        waveform = torchaudio.functional.resample(waveform, sr, target_sr)

    # 3. Trim silence (VAD)
    waveform = torchaudio.functional.vad(waveform, sample_rate=target_sr, trigger_level=20)

    # 4. Peak normalization (GPU)
    waveform = torch.nn.functional.normalize(waveform, dim=-1)

    # 5. Fixed-length padding/cropping
    max_samples = target_sr * max_duration
    if waveform.size(-1) > max_samples:
        waveform = waveform[..., :max_samples]
    else:
        pad_amount = max_samples - waveform.size(-1)
        waveform = torch.nn.functional.pad(waveform, (0, pad_amount))

    # 6. Extract Log-Mel features (GPU)
    mel_transform = torchaudio.transforms.MelSpectrogram(
        sample_rate=target_sr,
        n_fft=400,
        hop_length=160,
        n_mels=80,
    ).to(device)

    mel_spec = mel_transform(waveform)
    log_mel = torch.log(mel_spec + 1e-6)  # (1, 80, T)

    return log_mel.squeeze(0).T  # (T, 80)

In [21]:
audio_directory = 'cv-valid-dev'
audio_files = [train_df.dataset.loc[train_df.indices[i], 'filename'] for i in range(len(train_df))]

In [22]:
features = []
for audio in audio_files:
  audio_feature = preprocess_for_rnnt_torchaudio(os.path.join(audio_directory, audio))
  features.append(audio_feature)

In [24]:
texts = [dev_df.loc[idx, 'text'] for idx in train_df.indices]

In [25]:
texts[:5]

['look at those shoulders',
 'i heard a faint movement under my feet',
 "without such love one's dreams would have no meaning",
 'two days later the merchant spoke to the boy about the display',
 'your husband must have been in a great hurry']

In [26]:
import re

In [27]:
def clean_text(text):
    # Keep original case - REMOVED .lower()
    text = text.strip()

    # Handle apostrophes/contractions carefully
    text = re.sub(r"([!\"'#$%&()*\+,-./:;<=>?@\\\[\]^_`{|}~])", r" \1 ", text)
    text = re.sub("[^A-Za-z0-9]+", " ", text)
    text = re.sub(" +", " ", text)

    return text

In [28]:
# Apply cleaning
cleaned_texts = [clean_text(t) for t in texts if isinstance(t, str)]

In [29]:
cleaned_texts[:5]

['look at those shoulders',
 'i heard a faint movement under my feet',
 'without such love one s dreams would have no meaning',
 'two days later the merchant spoke to the boy about the display',
 'your husband must have been in a great hurry']

In [30]:
from tokenizers import Tokenizer, models, trainers
from tokenizers import pre_tokenizers

In [31]:
# Initialize with byte-level BPE (better for names/contractions)
tokenizer = Tokenizer(models.BPE(unk_token="<unk>"))

# Simpler pre-tokenizer (preserve apostrophes)
tokenizer.pre_tokenizer = pre_tokenizers.Sequence([
    pre_tokenizers.Whitespace(),
    pre_tokenizers.Split("'", behavior="isolated")  # Special handling for apostrophes
])

# Trainer with larger vocab
trainer = trainers.BpeTrainer(
    special_tokens=["<pad>", "<unk>", "<bos>", "<eos>"],
    vocab_size=3800,  # Increased for better word coverage
    min_frequency=3,
    initial_alphabet=pre_tokenizers.ByteLevel.alphabet()  # Better for special characters
)

# Train on properly cleaned text
tokenizer.train_from_iterator([clean_text(t) for t in texts], trainer)
tokenizer.save("cv_tokenizer.json")

In [32]:
# Load tokenizer
tokenizer = Tokenizer.from_file("cv_tokenizer.json")

# Test encoding
sample_text = "Dr O'Neill 's patient said We 'll check again tomorrow"
encoding = tokenizer.encode(sample_text.lower())
print("Tokens:", encoding.tokens)
print("IDs:", encoding.ids)

Tokens: ['dr', 'o', "'", 'ne', 'ill', "'", 's', 'pat', 'ient', 'said', 'we', "'", 'll', 'che', 'ck', 'again', 'tomorrow']
IDs: [921, 82, 10, 342, 474, 10, 86, 1310, 1392, 331, 297, 10, 296, 370, 455, 658, 1476]


In [82]:
class Encoder(torch.nn.Module):
    def __init__(self, input_dim=80, hidden_dim=256, num_layers=3):
        super().__init__()
        self.lstm = torch.nn.LSTM(
            input_size=input_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            bidirectional=True,
            batch_first=True,
        )
        self.linear = torch.nn.Linear(hidden_dim * 2, hidden_dim)

    def forward(self, x):
        x, _ = self.lstm(x)  # (B, T, 2*H)
        x = self.linear(x)    # (B, T, H)
        return x

In [83]:
class Decoder(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim=128, hidden_dim=256):
        super().__init__()
        self.embed = torch.nn.Embedding(vocab_size, embed_dim)
        self.lstm = torch.nn.LSTM(embed_dim, hidden_dim, batch_first=True)

    def forward(self, y):
        y = self.embed(y)     # (B, U, E)
        y, _ = self.lstm(y)   # (B, U, H)
        return y

In [84]:
class JointNetwork(torch.nn.Module):
    def __init__(self, hidden_dim, vocab_size):
        super().__init__()
        self.linear = torch.nn.Linear(hidden_dim, vocab_size)

    def forward(self, h_enc, h_dec):
        # h_enc: (B, T, H), h_dec: (B, U, H)
        h_enc = h_enc.unsqueeze(2)  # (B, T, 1, H)
        h_dec = h_dec.unsqueeze(1)  # (B, 1, U, H)
        out = torch.tanh(h_enc + h_dec)  # (B, T, U, H)
        return self.linear(out)  # (B, T, U, V)

In [106]:
class RNNTransducer(torch.nn.Module):
    def __init__(self, vocab_size, encoder_dim=256, decoder_dim=256):
        super().__init__()
        self.encoder = Encoder(hidden_dim=encoder_dim)
        self.decoder = Decoder(vocab_size, hidden_dim=decoder_dim)
        self.joint = JointNetwork(encoder_dim, vocab_size)

    def forward(self, x, y):
        h_enc = self.encoder(x)  # (B, T, H)
        h_dec = self.decoder(y)  # (B, U, H)
        logits = self.joint(h_enc, h_dec)  # (B, T, U, V)
        return logits

In [86]:
# Continue from where the notebook left off

# First, let's prepare the dataset class
class AudioTextDataset(torch.utils.data.Dataset):
    def __init__(self, features, texts, tokenizer, max_text_length=100):
        self.features = features
        self.texts = texts
        self.tokenizer = tokenizer
        self.max_text_length = max_text_length

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        # Get audio features
        audio_feature = self.features[idx]

        # Get and tokenize text
        text = self.texts[idx]
        encoding = self.tokenizer.encode(text)
        text_ids = encoding.ids

        # Add BOS and EOS tokens
        text_ids = [tokenizer.token_to_id("<bos>")] + text_ids + [tokenizer.token_to_id("<eos>")]

        # Pad text sequence
        if len(text_ids) < self.max_text_length:
            pad_amount = self.max_text_length - len(text_ids)
            text_ids = text_ids + [tokenizer.token_to_id("<pad>")] * pad_amount
        else:
            text_ids = text_ids[:self.max_text_length]

        return {
            'audio': audio_feature,
            'text': torch.tensor(text_ids, dtype=torch.long)
        }

In [44]:
# Create datasets
train_dataset = AudioTextDataset(
    features=features,
    texts=cleaned_texts,
    tokenizer=tokenizer
)

In [87]:
test_cleaned_texts = [clean_text(dev_df.loc[idx, 'text']) for idx in test_df.indices]
test_features = []
audio_directory = 'cv-valid-dev'
audio_files = [test_df.dataset.loc[test_df.indices[i], 'filename'] for i in range(len(test_df))]
for audio in audio_files:
  audio_feature = preprocess_for_rnnt_torchaudio(os.path.join(audio_directory, audio))
  test_features.append(audio_feature)

test_dataset = AudioTextDataset(
    features=test_features,
    texts=test_cleaned_texts,
    tokenizer=tokenizer
)

KeyboardInterrupt: 

In [88]:
# Create data loaders
def collate_fn(batch):
    # Pad audio features to same length
    audio_features = [item['audio'] for item in batch]
    max_audio_len = max(f.size(0) for f in audio_features)

    padded_audio = []
    for f in audio_features:
        pad_amount = max_audio_len - f.size(0)
        if pad_amount > 0:
            f = torch.nn.functional.pad(f, (0, 0, 0, pad_amount), value=0)
        padded_audio.append(f)

    audio_tensor = torch.stack(padded_audio)
    text_tensor = torch.stack([item['text'] for item in batch])

    return {
        'audio': audio_tensor.to(device),
        'text': text_tensor.to(device)
    }

batch_size = 2
train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn
)
test_loader = torch.utils.data.DataLoader(
    test_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn
)

In [108]:
# Initialize model
vocab_size = tokenizer.get_vocab_size()
model = RNNTransducer(vocab_size).to(device)

# Loss function (RNN-T loss)
def rnnt_loss(logits, targets, input_lengths, target_lengths, blank=0):
    # logits: (B, T, U, V)
    # targets: (B, U)
    # input_lengths: (B,) - length of each audio sequence
    # target_lengths: (B,) - length of each text sequence

    # Convert to log-probs
    log_probs = torch.nn.functional.log_softmax(logits, dim=-1)

    # Transpose for torchaudio's expectation (T, B, U, V)
    log_probs = log_probs.permute(1, 0, 2, 3)

    logits = logits.contiguous()
    # Compute RNN-T loss
    loss = torchaudio.functional.rnnt_loss(
        log_probs,
        targets,
        input_lengths,
        target_lengths,
        blank=blank,
        reduction='mean'
    )

    return loss


In [101]:
# Optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

# Training loop
def train_epoch(model, dataloader, optimizer, accumulation_steps=4):
    model.train()
    total_loss = 0

    for batch in dataloader:
        audio = batch['audio']
        text = batch['text']

        # Get input/target lengths
        input_lengths = torch.tensor([a.size(0) for a in audio], dtype=torch.int32, device=device)
        target_lengths = torch.tensor([(t != tokenizer.token_to_id("<pad>")).sum() for t in text], dtype=torch.int32, device=device)

        # Forward pass
        optimizer.zero_grad()
        logits = model(audio, text[:, :-1]).contiguous()  # Exclude EOS token for decoder input

        # Compute loss
        loss = rnnt_loss(
            logits,
            text[:, 1:].type(torch.int32),  # Exclude BOS token for targets
            input_lengths,
            target_lengths - 1  # Subtract 1 because we excluded BOS
        )

        # Backward pass
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    return total_loss / len(dataloader)

In [102]:
# Evaluation loop
def evaluate(model, dataloader):
    model.eval()
    total_loss = 0

    with torch.no_grad():
        for batch in dataloader:
            audio = batch['audio']
            text = batch['text']

            input_lengths = torch.tensor([a.size(0) for a in audio], dtype=torch.int32, device=device)
            target_lengths = torch.tensor([(t != tokenizer.token_to_id("<pad>")).sum() for t in text], dtype=torch.int32, device=device)

            logits = model(audio, text[:, :-1]).contiguous()

            loss = rnnt_loss(
                logits,
                text[:, 1:],
                input_lengths,
                target_lengths - 1
            )

            total_loss += loss.item()

    return total_loss / len(dataloader)

In [109]:
# Training
num_epochs = 5
for epoch in range(num_epochs):
    train_loss = train_epoch(model, train_loader, optimizer)
    val_loss = evaluate(model, test_loader)

    print(f"Epoch {epoch+1}/{num_epochs}")
    print(f"Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f}")


OutOfMemoryError: CUDA out of memory. Tried to allocate 1.06 GiB. GPU 0 has a total capacity of 14.74 GiB of which 48.12 MiB is free. Process 7313 has 14.69 GiB memory in use. Of the allocated memory 13.39 GiB is allocated by PyTorch, and 1.17 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
# Inference function
def predict(model, audio_feature, tokenizer, max_decode_len=100):
    model.eval()

    # Add batch dimension
    audio_feature = audio_feature.unsqueeze(0).to(device)
    input_length = torch.tensor([audio_feature.size(1)], device=device)

    # Initialize with BOS token
    decoded = [tokenizer.token_to_id("<bos>")]

    with torch.no_grad():
        h_enc = model.encoder(audio_feature)  # (1, T, H)

        for _ in range(max_decode_len):
            # Get last predicted token
            last_token = torch.tensor([decoded[-1]], device=device).unsqueeze(0)

            # Decoder step
            h_dec = model.decoder(last_token)  # (1, 1, H)

            # Joint network
            logits = model.joint(h_enc, h_dec)  # (1, T, 1, V)
            log_probs = torch.nn.functional.log_softmax(logits.squeeze(2), dim=-1)  # (1, T, V)

            # Sum over time (approximate)
            scores = log_probs.sum(1)  # (1, V)
            next_token = scores.argmax(-1).item()

            # Stop if EOS is predicted
            if next_token == tokenizer.token_to_id("<eos>"):
                break

            decoded.append(next_token)

    # Convert to text
    tokens = tokenizer.decode(decoded[1:])  # Skip BOS
    return tokens


In [ ]:
# Test inference
test_idx = 0
test_audio = features[test_idx]
test_text = cleaned_texts[test_idx]

predicted_text = predict(model, test_audio, tokenizer)
print(f"Original: {test_text}")
print(f"Predicted: {predicted_text}")